## 多个 feas_iti 且最后一趟列车 id 相同的乘客

|           |     rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
| :-------- | ------: | -----: | ---------: | -----: | -------: | -------: | --------: |
| 93194730  | 1525794 |      2 | 1076103701 |      2 | 10301630 |    65071 |     65670 |
| 8610761   |  119014 |      6 | 1101102701 |      3 | 11002833 |    30045 |     30791 |
| 18578209  |  230428 |      6 | 1053108503 |      2 | 10401849 |    29715 |     30255 |
| 72855273  | 1321497 |      3 | 1110112802 |      1 | 10401844 |    60803 |     61537 |
| 105488240 | 1781873 |      2 | 1028106602 |      2 | 10401738 |    69396 |     69625 |
| 90908613  | 1485498 |      2 | 1023112302 |      2 | 10702370 |    64374 |     64780 |
| 91388272  | 1494286 |      1 | 1104111501 |      2 | 10702247 |    65070 |     65278 |
| 53621559  |  957712 |      6 | 1121102503 |      3 | 10401786 |    49174 |     50141 |
| 19767473  |  243641 |      1 | 1081103901 |      2 | 10702323 |    29509 |     31141 |
| 107978876 | 1880069 |      2 | 1129107002 |      1 | 10201226 |    73278 |     74154 |
| 106615427 | 1817923 |      1 | 1027110801 |      2 | 10301362 |    71471 |     73514 |
| 76737269  | 1359494 |      1 | 1099102101 |      2 | 10301421 |    64071 |     65266 |
| 100422001 | 1661221 |      5 | 1061108702 |      1 | 10100270 |    66558 |     66886 |
| 2408620   |   41409 |      2 | 1111106502 |      1 | 10401886 |    25727 |     25836 |
| 101105912 | 1675451 |      1 | 1110101301 |      1 | 10401914 |    66944 |     67031 |
| 92513072  | 1514419 |      3 | 1097110302 |      3 | 10200839 |    65302 |     65408 |
| 81121188  | 1412555 |      1 | 1061109504 |      2 | 10201199 |    63371 |     63480 |
| 108069499 | 1886032 |      1 | 1032108201 |      1 | 10200950 |    73568 |     75192 |
| 73737366  | 1331466 |      1 | 1086110901 |      2 | 10702414 |    61616 |     62107 |
| 48046191  |  811513 |      3 | 1076103602 |      3 | 10301265 |    42747 |     43084 |


In [1]:
from src import config
from src.utils import *
from src.globals import get_etd, get_ttd
from src.itinerary import cal_dis_all
from src.walk_time_dis_calculator import WalkTimeDisModel

config.load_config()

wtd = WalkTimeDisModel(get_etd(), get_ttd())
left = read_(config.CONFIG["results"]["left"], show_timer=False)

df = cal_dis_all(wtd=wtd, left=left)
df

[INFO] Reading file: results\egress\etd_1.csv
[INFO] Reading file: results\transfer\ttd_1.csv
[INFO] Reading file: results\network\pathvia.pkl
[INFO] Reading file: results\network\platform.csv
[INFO] Reading file: results\trajectory\left.pkl
[INFO] Calculating entry walk distribution for all itineraries...
[INFO] Reading file: data\AFC.pkl
[INFO] Calculating egress walk distribution for all itineraries...
[INFO] Calculating transfer walk distribution for all itineraries...


,rid,iti_id,path_id,seg_id,t1,t2,pp_id1,pp_id2,time,dis
0,285,14,1120101002,1,21238,23410,103801,103802,2172,1.000000
1,285,14,1120101002,2,24412,27272,102201,102202,2860,1.000000
2,285,15,1120101002,1,21238,23410,103801,103802,2172,1.000000
3,285,15,1120101002,2,24412,24520,102201,102202,108,0.178590
4,285,16,1120101002,1,21238,23410,103801,103802,2172,1.000000
...,...,...,...,...,...,...,...,...,...,...
62976911,2046865,2,1097101501,-1,85823,85947,0,101501,124,0.298703
62976912,2046868,1,1097101501,-1,85481,85940,0,101501,459,0.000000
62976913,2046868,2,1097101501,-1,85823,85940,0,101501,117,0.358818
62976914,2046883,1,1129101001,-1,85741,85887,0,101002,146,0.857925


In [3]:
left['penalty'] = 1.0
left

,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts,penalty
2,285,1,1120101001,1,10702170,21132,22747,1.0
3,285,2,1120101001,1,10702170,25273,26962,1.0
4,285,3,1120101001,1,10702186,21797,23422,1.0
5,285,4,1120101001,1,10702186,25949,27655,1.0
6,285,5,1120101001,1,10702207,22652,24277,1.0
...,...,...,...,...,...,...,...,...
109949952,2046865,2,1097101501,1,10100645,85287,85823,1.0
109949955,2046868,1,1097101501,1,10100421,84945,85481,1.0
109949956,2046868,2,1097101501,1,10100645,85287,85823,1.0
109949965,2046883,1,1129101001,1,10702289,85531,85741,1.0
